In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import mnist
import keras
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Flatten, Dropout
from keras.models import Sequential
from keras.utils import to_categorical

In [ ]:
#training set
train_images = mnist.train_images()
train_labels = mnist.train_labels()

In [ ]:
print(train_images.shape)
print(train_labels.shape)


In [ ]:
#Test set
test_images = mnist.test_images()
test_labels = mnist.test_labels()
print(test_images.shape)

In [ ]:
#normalize the image pixel values from [0,255] to [-0.5,0.5] 
#to make our network easier to train (using smaller, centered values usually
#leads to better results)

train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

In [ ]:
# reshape each image from from (28,28) to (28,28,1) because 
#keras requires the third dimension

train_images = np.expand_dims(train_images,axis = 3)
test_images = np.expand_dims(test_images,axis = 3)

In [ ]:
print(train_images.shape)
print(test_images.shape)

In [ ]:
num_filter = 8
filter_size = 3
pool_size = 2

model = Sequential([Conv2D(num_filter,filter_size, input_shape=(28,28,1)),
                   MaxPooling2D(pool_size = pool_size),
                   Flatten(), 
                   Dense(10, activation='softmax')])

In [ ]:
model.compile('adam', loss = 'categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
#Train the model
model.fit(train_images, to_categorical(train_labels),
         epochs=5, validation_data=(test_images, to_categorical(test_labels)))

In [ ]:
import pathlib
import PIL

data_dir = pathlib.Path('../input/white-numbers/Num_images')
data_dir

In [ ]:
images = list(data_dir.glob('*/*.png'))


for i in range(len(images)):
    im = PIL.Image.open(str(images[i]))
    images[i] = im.resize((28, 28))
    
images[4]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=(1./255)- 0.5)
test_datagen

tst_ds = test_datagen.flow_from_directory(
                    directory= '../input/white-numbers/Num_images',
                    target_size=(28, 28),
                    color_mode="grayscale",
                    classes=None,
                    class_mode="categorical",
                    batch_size=5,
                    shuffle=False,
                    seed=None,
                    save_to_dir=None,
                    save_prefix="",
                    save_format="png",
                    follow_links=False,
                    subset=None,
                    interpolation="nearest",
                )

In [ ]:
tst_ds.class_indices.keys()

In [ ]:
tst_ds.labels

In [ ]:
preds = model.predict_generator(tst_ds)
preds

# print(len(preds))

In [ ]:
print (np.argmax(preds[6]))

In [ ]:
import cv2
import numpy as np

img = cv2.imread('../input/test-num/test.png')
img = cv2.resize(img,(28,28))
# img = np.reshape(img,[1,28,28,1])

classes = model.predict_classes(img)